In [1]:
import keras
import cv2
import skimage
import skimage.transform
import os
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.utils.training_utils import multi_gpu_model
%matplotlib inline

c:\users\vincenzo\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# We define in advance constants to build the model

INPUT_SHAPE = (96, 96, 3)
IM_HEIGHT = 96
IM_WIDTH = 96
OUTPUT_SIZE = 2

LEARNING_RATE = 0.01
OPTIMIZER = keras.optimizers.Adam()
LOSS = 'binary_crossentropy'
METRIC = 'accuracy'

SL_TRAIN_SIZE = 50121
SL_VALIDATION_SIZE = 11631
EPOCHS = 10
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 50
STEPS_PER_EPOCH = (2 * SL_TRAIN_SIZE) // BATCH_SIZE + 1
VALIDATION_STEPS_PER_EPOCH = (2 * SL_VALIDATION_SIZE) // BATCH_SIZE + 1

TRAIN_PATH = "./data_set/train/"
VALIDATION_PATH = "./data_set/validation/"
TEST_PATH = "./data_set/test/"

MODEL_PATH = "./binary_classifier/net_1_model.json"
WEIGHTS_PATH = "./binary_classifier/net_1_weights.h5"

In [3]:
def transform_image(img):
    #trasformazioni da implementare rotate, flip, shift, zoom
    sel = [0, 1]
    offset = 72
    t_img = img
    if rand.choice(sel):
        # rotate
        angle = rand.uniform(0, 360)
        t_img = skimage.transform.rotate(t_img, angle)
    if rand.choice(sel):
        # flip
        if rand.choice(sel):
            t_img = skimage.flipud(t_img)
        if rand.choice(sel):
            t_img = skimage.fliplr(t_img)    
    if rand.choice(sel):
        # zoom
        return skimage.resize(t_img, (96, 96), anti_aliasing=True)
    if rand.choice(sel):
        # shift
        offset = rand.uniform(48,97)     
    return t_img[offset-48:offset+48, offset-48:offset+48, :]

In [4]:
# Sea lions patches extractor
def extract_sea_lion_patch(path, epoch, n_epochs):
    img = cv2.imread(path)
    if rand.uniform(0.0, 1.0) < (epoch/n_epochs):
        # Perform transformation
        return transform_image(img)
    patch = img[72-48:72+48, 72-48:72+48, :]
    return patch

In [5]:
def train_generator(epoch_tot, batch_size, num_steps):
    sl_lst_tmp = os.listdir(TRAIN_PATH + 'sea_lions')
    print(len(sl_lst_tmp))
    sl_lst = []
    for elem in sl_lst_tmp:
        sl_lst.append(list((elem, 'sea_lion')))
    print(len(sl_lst))
    bkg_lst_tmp = os.listdir(TRAIN_PATH + 'background')
    print(len(bkg_lst_tmp))
    for curr_epoch in range(epoch_tot):
        bkg_lst = []
        for elem in rand.sample(bkg_lst_tmp, SL_TRAIN_SIZE):
            bkg_lst.append(list((elem, 'background')))
        print(len(bkg_lst))
        lst = sl_lst + bkg_lst
        rand.shuffle(lst)
        print(len(lst))
        for step in range(num_steps):
            patches = []
            classes = []
            for i in range(step*batch_size, min((step*batch_size)+batch_size, 2*SL_TRAIN_SIZE)):
                if lst[(step*batch_size)+i][1] == 'background':
                    img = cv2.imread(TRAIN_PATH + 'background/' + lst[(step*batch_size)+i][0])
                    patches.append(img)
                    classes.append([0, 1])
                else:
                    patches.append(extract_sea_lion_patch(TRAIN_PATH + 'sea_lions/' + lst[(step*batch_size)+i][0], curr_epoch, epoch_tot))
                    classes.append([1, 0])
            X_train = np.array(patches)
            X_train = X_train.astype('float32')
            X_train /= 255
            Y_train = np.array(classes)
            yield X_train, Y_train

In [7]:
b = 0
for a in train_generator(10, 50, STEPS_PER_EPOCH):
    b += 1
    if b > 1:
        break

50121
50121


1323634
50121
100242


In [6]:
# Create data generators for the validation set

validation_data_gen = ImageDataGenerator(rescale=1./255)


validation_generator = validation_data_gen.flow_from_directory(
    VALIDATION_PATH,
    classes=['sea_lions', 'background'],
    target_size=(IM_HEIGHT, IM_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

Found 25062 images belonging to 2 classes.


In [7]:
# Build parallel model (multi gpu)

model = Sequential()
# First layer
model.add(Convolution2D(8, (5, 5), activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third layer
model.add(Convolution2D(5, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth layer
model.add(Convolution2D(10, (3, 3), activation='relu', padding='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(OUTPUT_SIZE, activation='softmax'))

parallel_model = multi_gpu_model(model, gpus=2)
parallel_model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=[METRIC])

In [8]:
# Train (multi gpu)

# Fit model on training data
history = parallel_model.fit_generator( 
    train_generator(EPOCHS, BATCH_SIZE, STEPS_PER_EPOCH),
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    verbose=1,
    validation_data=validation_generator, 
    validation_steps=VALIDATION_STEPS_PER_EPOCH,
    workers=8,
    max_queue_size=50)

Epoch 1/10


   1/2005 [..............................] - ETA: 2:38:07 - loss: 0.6854 - acc: 0.5200

   2/2005 [..............................] - ETA: 1:33:53 - loss: 0.6990 - acc: 0.4900

   3/2005 [..............................] - ETA: 1:22:12 - loss: 0.6938 - acc: 0.5000

   4/2005 [..............................] - ETA: 1:15:41 - loss: 0.6900 - acc: 0.5200

...

 999/2005 [=============>................] - ETA: 10:03 - loss: 0.3130 - acc: 0.8649

1000/2005 [=============>................] - ETA: 10:03 - loss: 0.3129 - acc: 0.8650

1001/2005 [=============>................] - ETA: 10:02 - loss: 0.3127 - acc: 0.8651

1002/2005 [=============>................] - ETA: 10:01 - loss: 0.3128 - acc: 0.8650

IndexError: list index out of range

In [15]:
# Summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [11]:
# Save trained model (multi gpu)

# serialize weights to HDF5
parallel_model.save_weights(WEIGHTS_PATH)

In [ ]:
# TODO add testing